**Big Data - Lab Spark SQL**

**DOULADI Oussama**

**SupMTI-3GI-DS**

L’objectif de ce lab est de se familiariser avec l’api spark sql.

In [148]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Spark SQL Lab - Football") \
    .getOrCreate()

df = spark.read.csv(
    "fifaworldcup.csv",
    header=True,
    inferSchema=True
)

df.createOrReplaceTempView("matches")

#afficher un aperçu des données
spark.sql("SELECT * FROM matches LIMIT 5").show()



+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  false|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  false|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  false|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  false|
|1876-03-04| Scotland|  England|         3|         0|  Friendly|Glasgow|Scotland|  false|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+



1. Nombre total de matchs

In [ ]:
#df.count()
spark.sql("""
SELECT COUNT(*) AS total_matches
FROM matches
""").show()


+-------------+
|total_matches|
+-------------+
|        44152|
+-------------+



2. Première et dernière année

In [116]:
spark.sql("""
SELECT MIN(YEAR(date)) AS first_year,
       MAX(YEAR(date)) AS last_year
FROM matches
""").show()



+----------+---------+
|first_year|last_year|
+----------+---------+
|      1872|     2022|
+----------+---------+

+----------+---------+
|first_year|last_year|
+----------+---------+
|      1872|     2022|
+----------+---------+



3. Top 10 des tournois les plus fréquents

In [117]:
spark.sql("""
SELECT tournament, COUNT(*) AS nb_matches
FROM matches
GROUP BY tournament
ORDER BY nb_matches DESC
LIMIT 10
""").show()


+--------------------+----------+
|          tournament|nb_matches|
+--------------------+----------+
|            Friendly|     17461|
|FIFA World Cup qu...|      7774|
|UEFA Euro qualifi...|      2593|
|African Cup of Na...|      1932|
|      FIFA World Cup|       948|
|        Copa América|       841|
|AFC Asian Cup qua...|       764|
|African Cup of Na...|       742|
|          CECAFA Cup|       620|
|CFU Caribbean Cup...|       606|
+--------------------+----------+



4. Matchs joués sur terrain neutre

In [118]:
spark.sql("""
SELECT COUNT(*) AS neutral_matches
FROM matches
WHERE neutral = true
""").show()



+---------------+
|neutral_matches|
+---------------+
|          10996|
+---------------+



5. Top 10 des pays ayant accueilli le plus de matchs

In [119]:
spark.sql("""
SELECT country, COUNT(*) AS nb_matches
FROM matches
GROUP BY country
ORDER BY nb_matches DESC
LIMIT 10
""").show()


+--------------------+----------+
|             country|nb_matches|
+--------------------+----------+
|       United States|      1259|
|              France|       830|
|            Malaysia|       752|
|             England|       722|
|              Sweden|       659|
|               Qatar|       618|
|             Germany|       608|
|              Brazil|       569|
|               Spain|       568|
|United Arab Emirates|       548|
+--------------------+----------+



6. Matchs terminés par un nul

In [120]:
spark.sql("""
SELECT COUNT(*) AS draws
FROM matches
WHERE home_score = away_score
""").show()


+-----+
|draws|
+-----+
|10213|
+-----+



7. Matchs avec score total > 6

In [121]:
spark.sql("""
SELECT *
FROM matches
WHERE CAST(home_score AS INT) + CAST(away_score AS INT) > 6
""").show()



+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|      date|       home_team|       away_team|home_score|away_score|          tournament|     city| country|neutral|
+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|1878-03-02|        Scotland|         England|         7|         2|            Friendly|  Glasgow|Scotland|  false|
|1878-03-23|        Scotland|           Wales|         9|         0|            Friendly|  Glasgow|Scotland|  false|
|1879-04-05|         England|        Scotland|         5|         4|            Friendly|   London| England|  false|
|1880-03-13|        Scotland|         England|         5|         4|            Friendly|  Glasgow|Scotland|  false|
|1881-03-12|         England|        Scotland|         1|         6|            Friendly|   London| England|  false|
|1882-02-18|Northern Ireland|         England|         0|       

8. Nombre total de matchs par équipe

In [122]:
spark.sql("""
SELECT team, COUNT(*) AS total_matches
FROM (
    SELECT home_team AS team FROM matches
    UNION ALL
    SELECT away_team AS team FROM matches
)
GROUP BY team
ORDER BY total_matches DESC
""").show()


+-----------+-------------+
|       team|total_matches|
+-----------+-------------+
|     Sweden|         1052|
|    England|         1047|
|     Brazil|         1019|
|  Argentina|         1014|
|    Germany|          986|
|    Hungary|          964|
|     Mexico|          926|
|    Uruguay|          919|
|South Korea|          904|
|     France|          875|
|     Poland|          851|
|      Italy|          837|
|Switzerland|          834|
|     Norway|          832|
|    Denmark|          832|
|    Austria|          820|
|Netherlands|          820|
|   Scotland|          814|
|    Belgium|          804|
|      Chile|          793|
+-----------+-------------+
only showing top 20 rows


9. Top 10 des équipes ayant marqué le plus de buts

In [123]:
spark.sql("""
SELECT team, SUM(TRY_CAST(goals AS INT)) AS total_goals
FROM (
    SELECT home_team AS team, home_score AS goals FROM matches
    UNION ALL
    SELECT away_team AS team, away_score AS goals FROM matches
)
GROUP BY team
ORDER BY total_goals DESC
LIMIT 10
""").show()


+-----------+-----------+
|       team|total_goals|
+-----------+-----------+
|    England|       2282|
|     Brazil|       2228|
|    Germany|       2205|
|     Sweden|       2060|
|    Hungary|       1948|
|  Argentina|       1880|
|Netherlands|       1694|
|     Mexico|       1592|
|South Korea|       1576|
|     France|       1560|
+-----------+-----------+



10. Moyenne de buts par match par décennie

In [124]:
spark.sql("""
SELECT (YEAR(date) / 10) * 10 AS decade,
       ROUND(AVG(try_cast(home_score as int) + try_cast(away_score as int)), 2) AS avg_goals
FROM matches
GROUP BY (YEAR(date) / 10) * 10
ORDER BY decade
""").show()


+------+---------+
|decade|avg_goals|
+------+---------+
|1872.0|      0.0|
|1873.0|      6.0|
|1874.0|      3.0|
|1875.0|      4.0|
|1876.0|      3.5|
|1877.0|      3.0|
|1878.0|      9.0|
|1879.0|      5.0|
|1880.0|     6.67|
|1881.0|     4.67|
|1882.0|      8.0|
|1883.0|      4.4|
|1884.0|      5.0|
|1885.0|     5.43|
|1886.0|     5.29|
|1887.0|     4.67|
|1888.0|     7.14|
|1889.0|     4.67|
|1890.0|      5.5|
|1891.0|     5.67|
+------+---------+
only showing top 20 rows


11. Matchs joués par tournoi et par année

In [125]:
spark.sql("""
SELECT tournament,
       YEAR(date) AS year,
       COUNT(*) AS nb_matches
FROM matches
GROUP BY tournament, YEAR(date)
ORDER BY year, nb_matches DESC
""").show()


+--------------------+----+----------+
|          tournament|year|nb_matches|
+--------------------+----+----------+
|            Friendly|1872|         1|
|            Friendly|1873|         1|
|            Friendly|1874|         1|
|            Friendly|1875|         1|
|            Friendly|1876|         2|
|            Friendly|1877|         2|
|            Friendly|1878|         2|
|            Friendly|1879|         3|
|            Friendly|1880|         3|
|            Friendly|1881|         3|
|            Friendly|1882|         5|
|            Friendly|1883|         5|
|British Championship|1884|         6|
|British Championship|1885|         6|
|            Friendly|1885|         1|
|British Championship|1886|         6|
|            Friendly|1886|         1|
|British Championship|1887|         6|
|British Championship|1888|         6|
|            Friendly|1888|         1|
+--------------------+----+----------+
only showing top 20 rows


12. Classement des équipes ayant gagné le plus à domicile

In [126]:
spark.sql("""
SELECT home_team AS team, COUNT(*) AS home_wins
FROM matches
WHERE home_score > away_score
GROUP BY home_team
ORDER BY home_wins DESC
""").show()


+-------------+---------+
|         team|home_wins|
+-------------+---------+
|       Brazil|      423|
|    Argentina|      373|
|      Germany|      327|
|      England|      323|
|       Mexico|      320|
|  South Korea|      296|
|       Sweden|      294|
|        Italy|      291|
|       France|      290|
|      Hungary|      265|
|        Egypt|      260|
|        Spain|      256|
|  Netherlands|      252|
|United States|      243|
|      Belgium|      229|
|      Denmark|      229|
|     Scotland|      224|
| Saudi Arabia|      222|
|      Austria|      220|
|        Chile|      213|
+-------------+---------+
only showing top 20 rows


13. Victoires / Défaites / Nuls par équipe

In [132]:
spark.sql("""
SELECT team,
       SUM(win) AS victoires,
       SUM(loss) AS defaites,
       SUM(draw) AS nuls
FROM (
    SELECT home_team AS team,
           CASE WHEN home_score > away_score THEN 1 ELSE 0 END AS win,
           CASE WHEN home_score < away_score THEN 1 ELSE 0 END AS loss,
           CASE WHEN home_score = away_score THEN 1 ELSE 0 END AS draw
    FROM matches
    UNION ALL
    SELECT away_team AS team,
           CASE WHEN away_score > home_score THEN 1 ELSE 0 END,
           CASE WHEN away_score < home_score THEN 1 ELSE 0 END,
           CASE WHEN away_score = home_score THEN 1 ELSE 0 END
    FROM matches
)
GROUP BY team
ORDER BY victoires DESC
""").show()


+-----------+---------+--------+----+
|       team|victoires|defaites|nuls|
+-----------+---------+--------+----+
|     Brazil|      651|     160| 208|
|    England|      593|     201| 253|
|    Germany|      573|     205| 208|
|  Argentina|      547|     213| 254|
|     Sweden|      516|     309| 227|
|South Korea|      477|     192| 235|
|     Mexico|      467|     243| 216|
|    Hungary|      452|     301| 211|
|      Italy|      445|     157| 235|
|     France|      436|     249| 190|
|      Spain|      424|     134| 173|
|Netherlands|      419|     213| 188|
|    Uruguay|      399|     294| 226|
|   Scotland|      385|     253| 176|
|    Denmark|      378|     278| 176|
|     Russia|      365|     164| 187|
|     Poland|      365|     268| 218|
|    Belgium|      354|     277| 173|
|     Zambia|      346|     219| 201|
|    Austria|      340|     303| 177|
+-----------+---------+--------+----+
only showing top 20 rows


14. Score moyen terrain neutre vs non neutre

In [134]:
spark.sql("""
SELECT neutral,
       round(AVG(try_cast(home_score as int) + try_cast(away_score as int)),3) AS score_moyen
FROM matches
GROUP BY neutral
""").show()


+-------+-----------+
|neutral|score_moyen|
+-------+-----------+
|   true|      3.015|
|  false|      2.886|
+-------+-----------+



15. Top 5 des matchs avec plus grand écart de score

In [135]:
spark.sql("""
SELECT *,
       ABS(try_cast(home_score as int) - try_cast(away_score as int)) AS goal_diff
FROM matches
ORDER BY goal_diff DESC
LIMIT 5
""").show()


+----------+---------+--------------+----------+----------+--------------------+-------------+----------------+-------+---------+
|      date|home_team|     away_team|home_score|away_score|          tournament|         city|         country|neutral|goal_diff|
+----------+---------+--------------+----------+----------+--------------------+-------------+----------------+-------+---------+
|2001-04-11|Australia|American Samoa|        31|         0|FIFA World Cup qu...|Coffs Harbour|       Australia|  false|       31|
|1971-09-13|   Tahiti|  Cook Islands|        30|         0| South Pacific Games|      Papeete|French Polynesia|  false|       30|
|1979-08-30|     Fiji|      Kiribati|        24|         0| South Pacific Games|      Nausori|            Fiji|  false|       24|
|2001-04-09|Australia|         Tonga|        22|         0|FIFA World Cup qu...|Coffs Harbour|       Australia|  false|       22|
|1966-04-03|    Libya|          Oman|        21|         0|            Arab Cup|      Bagh

16. Goal average par équipe

In [136]:
spark.sql("""
SELECT team,
       ROUND(SUM(scored_val) - SUM(conceded_val), 2) AS goal_average
FROM (
    SELECT
        home_team AS team,
        CASE WHEN home_score != 'NA' THEN CAST(home_score AS DOUBLE) ELSE NULL END AS scored_val,
        CASE WHEN away_score != 'NA' THEN CAST(away_score AS DOUBLE) ELSE NULL END AS conceded_val
    FROM matches

    UNION ALL

    SELECT
        away_team AS team,
        CASE WHEN away_score != 'NA' THEN CAST(away_score AS DOUBLE) ELSE NULL END AS scored_val,
        CASE WHEN home_score != 'NA' THEN CAST(home_score AS DOUBLE) ELSE NULL END AS conceded_val
    FROM matches
)
GROUP BY team
ORDER BY goal_average DESC
""").show()


+-----------+------------+
|       team|goal_average|
+-----------+------------+
|     Brazil|      1318.0|
|    England|      1268.0|
|    Germany|      1072.0|
|  Argentina|       838.0|
|      Spain|       814.0|
|South Korea|       768.0|
|     Sweden|       713.0|
|Netherlands|       694.0|
|      Italy|       638.0|
|     Mexico|       616.0|
|       Iran|       556.0|
|     Russia|       533.0|
|    Hungary|       519.0|
|  Australia|       498.0|
|     France|       432.0|
|   China PR|       423.0|
|      Egypt|       416.0|
|      Japan|       404.0|
|   Scotland|       398.0|
|     Zambia|       396.0|
+-----------+------------+
only showing top 20 rows


17. Classement des équipes par victoires par année

In [137]:
spark.sql("""
SELECT year, team, wins,
       ROW_NUMBER() OVER (PARTITION BY year ORDER BY wins DESC) AS rank
FROM (
    SELECT YEAR(date) AS year,
           home_team AS team,
           COUNT(*) AS wins
    FROM matches
    WHERE home_score > away_score
    GROUP BY YEAR(date), home_team
)
""").show()


+----+----------------+----+----+
|year|            team|wins|rank|
+----+----------------+----+----+
|1873|         England|   1|   1|
|1874|        Scotland|   1|   1|
|1876|        Scotland|   2|   1|
|1878|        Scotland|   2|   1|
|1879|         England|   2|   1|
|1880|        Scotland|   2|   1|
|1882|        Scotland|   2|   1|
|1882|           Wales|   2|   2|
|1883|         England|   2|   1|
|1884|        Scotland|   2|   1|
|1884|           Wales|   1|   2|
|1885|        Scotland|   1|   1|
|1885|         England|   1|   2|
|1886|        Scotland|   1|   1|
|1886|           Wales|   1|   2|
|1886|   United States|   1|   3|
|1887|         England|   2|   1|
|1887|        Scotland|   1|   2|
|1887|Northern Ireland|   1|   3|
|1888|        Scotland|   2|   1|
+----+----------------+----+----+
only showing top 20 rows


18. Évolution du nombre de matchs par décennie

In [141]:
spark.sql("""
SELECT
  (YEAR(date) DIV 10) * 10 AS decade,
  COUNT(*) AS nb_matches
FROM matches
GROUP BY (YEAR(date) DIV 10) * 10
ORDER BY decade
""").show()


+------+----------+
|decade|nb_matches|
+------+----------+
|  1870|        13|
|  1880|        55|
|  1890|        59|
|  1900|       124|
|  1910|       283|
|  1920|       729|
|  1930|      1010|
|  1940|       804|
|  1950|      1576|
|  1960|      2810|
|  1970|      3922|
|  1980|      4795|
|  1990|      6596|
|  2000|      9422|
|  2010|      9735|
|  2020|      2219|
+------+----------+



19. Équipes invaincues sur une année

In [142]:
spark.sql("""
SELECT team, year
FROM (
    SELECT team, year, SUM(loss) AS losses
    FROM (
        SELECT home_team AS team,
               YEAR(date) AS year,
               CASE WHEN home_score < away_score THEN 1 ELSE 0 END AS loss
        FROM matches
        UNION ALL
        SELECT away_team,
               YEAR(date),
               CASE WHEN away_score < home_score THEN 1 ELSE 0 END
        FROM matches
    )
    GROUP BY team, year
)
WHERE losses = 0
""").show()


+------------------+----+
|              team|year|
+------------------+----+
|            Jersey|2009|
|         Guatemala|2020|
|          Scotland|1890|
|             Kenya|1953|
|             Egypt|1958|
|             Gabon|2007|
|Dominican Republic|2017|
|           Curaçao|1934|
|            Sweden|1940|
|         German DR|1955|
|             Haiti|1971|
|             Haiti|1979|
|         Catalonia|2003|
|            Mexico|1949|
|           Romania|1943|
|             Italy|1946|
|     Guinea-Bissau|1987|
|            Jersey|1996|
|            Jordan|1965|
|          Suriname|1967|
+------------------+----+
only showing top 20 rows


20. Calculer la longue série de victoires consécutives par équipe.

In [145]:
# Créer une vue avec les victoires par équipe
spark.sql("""
CREATE OR REPLACE TEMP VIEW team_matches AS
SELECT home_team AS team, date,
       CASE WHEN home_score > away_score THEN 1 ELSE 0 END AS is_win
FROM matches
UNION ALL
SELECT away_team AS team, date,
       CASE WHEN away_score > home_score THEN 1 ELSE 0 END AS is_win
FROM matches
""")

# Ajouter un rang chronologique par équipe
spark.sql("""
CREATE OR REPLACE TEMP VIEW team_matches_ranked AS
SELECT *,
       ROW_NUMBER() OVER (PARTITION BY team ORDER BY date) AS rn
FROM team_matches
""")

# Identifier les blocs de victoires consécutives
spark.sql("""
CREATE OR REPLACE TEMP VIEW team_streaks AS
SELECT *,
       SUM(CASE WHEN is_win = 0 THEN 1 ELSE 0 END) OVER
           (PARTITION BY team ORDER BY rn ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS streak_group
FROM team_matches_ranked
""")

# Calculer la longueur de chaque streak
spark.sql("""
CREATE OR REPLACE TEMP VIEW streak_lengths AS
SELECT team, streak_group, SUM(is_win) AS streak_length
FROM team_streaks
GROUP BY team, streak_group
""")

# Trouver la plus longue série par équipe
longest_streak = spark.sql("""
SELECT team, MAX(streak_length) AS serie_plus_longue
FROM streak_lengths
GROUP BY team
ORDER BY serie_plus_longue DESC
""")

# Afficher le top 10 des plus longues séries
longest_streak.show(10, truncate=False)

+---------+-----------------+
|team     |serie_plus_longue|
+---------+-----------------+
|Padania  |15               |
|Spain    |15               |
|France   |14               |
|Brazil   |14               |
|Australia|13               |
|Guyana   |13               |
|Italy    |13               |
|Mexico   |13               |
|Scotland |13               |
|German DR|12               |
+---------+-----------------+
only showing top 10 rows


21. Équipe la plus victorieuse par tournoi

In [146]:
spark.sql("""
SELECT tournament, team, wins
FROM (
    SELECT tournament, team, COUNT(*) AS wins,
           ROW_NUMBER() OVER (PARTITION BY tournament ORDER BY COUNT(*) DESC) AS rank
    FROM (
        SELECT tournament, home_team AS team
        FROM matches
        WHERE home_score > away_score
        UNION ALL
        SELECT tournament, away_team
        FROM matches
        WHERE away_score > home_score
    )
    GROUP BY tournament, team
)
WHERE rank = 1
""").show()


+--------------------+------------------+----+
|          tournament|              team|wins|
+--------------------+------------------+----+
|     ABCS Tournament|          Suriname|   7|
|       AFC Asian Cup|              Iran|  41|
|AFC Asian Cup qua...|              Iran|  36|
|   AFC Challenge Cup|        Tajikistan|  11|
|AFC Challenge Cup...|       Philippines|   7|
|    AFF Championship|          Thailand|  42|
|AFF Championship ...|       Timor-Leste|   2|
|African Cup of Na...|             Egypt|  60|
|African Cup of Na...|       Ivory Coast|  67|
|African Nations C...|           Morocco|  11|
|African Nations C...|            Uganda|   4|
|    Afro-Asian Games|             India|   2|
|  Amílcar Cabral Cup|           Senegal|  38|
|            Arab Cup|              Iraq|  13|
|Arab Cup qualific...|           Lebanon|   3|
|        Atlantic Cup|         Argentina|   3|
|Atlantic Heritage...|Parishes of Jersey|   1|
|          Balkan Cup|           Romania|  21|
|          Ba

22. Performance domicile vs extérieur

In [147]:
spark.sql("""
SELECT team,
       SUM(home_win) AS home_wins,
       SUM(away_win) AS away_wins
FROM (
    SELECT home_team AS team,
           CASE WHEN home_score > away_score THEN 1 ELSE 0 END AS home_win,
           0 AS away_win
    FROM matches
    UNION ALL
    SELECT away_team,
           0,
           CASE WHEN away_score > home_score THEN 1 ELSE 0 END
    FROM matches
)
GROUP BY team
""").show()


+-------------+---------+---------+
|         team|home_wins|away_wins|
+-------------+---------+---------+
|      Kabylia|        0|        1|
|       Kernow|        3|        3|
|         Chad|       16|        7|
|     Provence|        2|        2|
|       Russia|      189|      176|
|     Paraguay|      133|      129|
|     Anguilla|        2|        1|
|        Yemen|       31|       13|
|South Ossetia|        1|        3|
|      Senegal|      150|      109|
|       Madrid|        0|        0|
|       Sweden|      294|      222|
|        Frøya|        3|        3|
|     Kiribati|        0|        0|
|     Ynys Môn|       18|       14|
|       Guyana|       54|       31|
|  Philippines|       45|       21|
|       Jersey|       38|       17|
|      Eritrea|        7|        4|
|        Tibet|        0|        0|
+-------------+---------+---------+
only showing top 20 rows
